<a href="https://colab.research.google.com/github/surasakcho/A_A/blob/master/01_03_colab_ls8_polygon_pixel_rowcol_d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libraries & Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install rasterio
!pip install geopandas


     |████████████████████████████████| 18.1MB 238kB/s 
     |████████████████████████████████| 931kB 2.7MB/s 
     |████████████████████████████████| 14.7MB 292kB/s 
     |████████████████████████████████| 10.4MB 30.3MB/s 


In [0]:
import numpy as np
import pandas as pd
import os, shutil
from glob import glob
import re
import datetime
from datetime import datetime
import random
from tqdm import tqdm_notebook
import pickle
import matplotlib.pyplot as plt
import geopandas as gpd
from pyproj import CRS
import rasterio
from rasterio import plot
from rasterio.transform import from_origin
from rasterio.warp import reproject, Resampling
from rasterio.mask import mask
from shapely.geometry import Polygon, mapping

%matplotlib inline

# Define functions

In [0]:

def list_files_re(rootpath, filename_re=None, folder_re=None ):
    '''
    rootpath : root path to lookup files
    filename_re : regular expression to search for filename
    folder_re : regular expression to search for folder

    return : a list of filepaths
    '''


    list_files = []
    for folder, _, files in os.walk(rootpath):
        for file in files:     
            if filename_re == None:
                filename_re = '.*'
            if folder_re == None:
                folder_re = '.*'
                
            if ((re.search(filename_re, file) != None) & (re.search(folder_re, folder) != None)):
                list_files.append(os.path.join(folder, file))
        
    return list_files    
        
        
    

In [0]:

def read_pickle(in_pickle_path):
    '''
    Read a pickle file.

    Parameters
    ----------------------------------
    in_pickle_path : path of pickle to be read

    Return
    ----------------------------------
    An object / variable of read pickle file
    '''

    pkl_file = open(in_pickle_path, 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()

    return data

# Define paths

In [0]:

my_drive_path = "/content/drive/My Drive/"

landsat_path = os.path.join(my_drive_path, '!landsat8-prep')
ndvi_path = os.path.join(landsat_path, 'ndvi')
vrt_path = os.path.join(landsat_path, 'vrt')
polygon_prov_path = os.path.join(landsat_path, 'plot_polygon_prov_dissolved')
pixel_rowcol_map_path = os.path.join(landsat_path, 'pixel_rowcol_mapping')


In [7]:
pixel_rowcol_map_path

'/content/drive/My Drive/!landsat8-prep/pixel_rowcol_mapping'

# Define parameters

In [0]:
crs_4326 = {'init' : 'epsg:4326'}
crs_47 = {'init' : 'epsg:32647'}
crs_48 = {'init' : 'epsg:32648'}

# Get files list

In [0]:
#vrt files
list_pixel_rowcol_map_files = list_files_re(pixel_rowcol_map_path, '.tif')
list_pixel_rowcol_map_files.sort()

#polygon prov files (pickle of gdf)
list_polygon_prov_files = list_files_re(polygon_prov_path, '.pkl')

#shuffle polygon list for multiple execution
random.shuffle(list_polygon_prov_files)
#list_polygon_prov_files.sort()



In [10]:
list_pixel_rowcol_map_files

['/content/drive/My Drive/!landsat8-prep/pixel_rowcol_mapping/ls8_pixel_rowcol_map_126049.tif',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_mapping/ls8_pixel_rowcol_map_126050.tif',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_mapping/ls8_pixel_rowcol_map_127047.tif',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_mapping/ls8_pixel_rowcol_map_127048.tif',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_mapping/ls8_pixel_rowcol_map_127049.tif',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_mapping/ls8_pixel_rowcol_map_127050.tif',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_mapping/ls8_pixel_rowcol_map_127051.tif',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_mapping/ls8_pixel_rowcol_map_127052.tif',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_mapping/ls8_pixel_rowcol_map_127055.tif',
 '/content/drive/My Drive/!landsat8-prep/pixel_rowcol_mapping/ls8_pixel_rowcol_map_127056.tif',
 '/content/drive/My Drive/!landsat8-prep

# Intersect pixel rowcol with plot polygon

In [0]:


for polygon_prov_file in tqdm_notebook(list_polygon_prov_files):
    prov_cd = int(polygon_prov_file.split('/')[-1].split('.')[0][-2:])
    print(f'prov_cd : {prov_cd} start')
    out_parquet_path = os.path.join(pixel_rowcol_map_path, f'df_pixel_rowcol_map_p{prov_cd}.parquet')

    #skip if file exists
    if os.path.exists(out_parquet_path):        
        continue

    list_df_intersect = []
    gdf = read_pickle(polygon_prov_file)

    for in_rst in list_pixel_rowcol_map_files:  
        scene_id = int(in_rst.split('/')[-1].split('.')[0][-6:])

        with rasterio.open(in_rst) as rst:
            rst_crs_str = rst.crs.to_string()
            
            #reproject polygon to same as raster
            gdf_curr = gdf.to_crs(rasterio.crs.CRS.from_string(rst_crs_str))

            assert(rst_crs_str == gdf_curr.crs.to_string())

            try:
                #intersect polygon and raster
                intersect_image, intersect_transform = mask(rst, gdf_curr['geometry'], crop=True, all_touched=True)
                print(f'Found {np.where(intersect_image[1] > 0, 1, 0).sum()} pixels in {scene_id}') 

                #get row & col from intersection result                   
                rows = intersect_image[0].flatten()
                rows = np.where(rows > 1, rows, np.nan)
                cols = intersect_image[1].flatten()
                cols = np.where(cols > 1, cols, np.nan)

                #create temp dataframe to collect row & col
                df_curr = pd.DataFrame(zip(rows, cols), columns=['row', 'col']).dropna()
                df_curr['row'] = df_curr['row'].astype(np.int16)
                df_curr['col'] = df_curr['col'].astype(np.int16)
                df_curr['prov_cd'] = prov_cd
                df_curr['scene_id'] = scene_id
                df_curr = df_curr.drop_duplicates()
                
                #append to list
                list_df_intersect.append(df_curr)
            except:
                pass
                #print(f'Not found in {in_rst}')

    df_pixel_map = pd.concat(list_df_intersect)
    if os.path.exists(out_parquet_path):        
        continue

    df_pixel_map.to_parquet(out_parquet_path)
    print(f'prov_cd : {prov_cd} done.')

    print()    

prov_cd : 70 start


/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Found 272102 pixels in 129050
Found 368038 pixels in 129051
Found 318254 pixels in 130050
Found 368070 pixels in 130051
prov_cd : 70 done.

prov_cd : 43 start


/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Found 1687 pixels in 127048
Found 763900 pixels in 128047
Found 846886 pixels in 128048
Found 270683 pixels in 129047
Found 81982 pixels in 129048
prov_cd : 43 done.

prov_cd : 13 start


...

Found 16764 pixels in 128050
Found 678822 pixels in 129049
Found 863251 pixels in 129050
Found 641850 pixels in 130049
Found 549906 pixels in 130050
prov_cd : 16 done.

prov_cd : 67 start


/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Found 3363 pixels in 128049
Found 13385 pixels in 128050
Found 638380 pixels in 129048
Found 1472790 pixels in 129049
Found 72647 pixels in 129050
Found 620843 pixels in 130048
Found 260788 pixels in 130049
Found 0 pixels in 130050
prov_cd : 67 done.

prov_cd : 90 start
prov_cd : 91 start
prov_cd : 36 start


/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Found 438477 pixels in 128048
Found 2172004 pixels in 128049
Found 112640 pixels in 128050
Found 415013 pixels in 129048
Found 1555974 pixels in 129049
Found 62551 pixels in 129050
